## First Part: Training the model (DistilBert)
### This notebook aims to develop a model to use in the Python service application

### Install dependencies (If needed) and import necesary packages

In [1]:
pip install mysql-connector-python transformers pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TextClassificationPipeline
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopw = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Extract data, previously downloaded

In [4]:
data_folder = '/content/drive/MyDrive/prueba/data/'
emails_df = pd.read_csv(data_folder + 'emails.csv')
clients_df = pd.read_csv(data_folder + 'clients.csv')
non_pay_df = pd.read_csv(data_folder + 'non_pay.csv')

In [5]:
# Apply styling to DataFrames
styled_emails = emails_df.style.set_caption("Emails DataFrame")
styled_clients = clients_df[:4].style.set_caption("Clients DataFrame")
styled_non_pay = non_pay_df[:4].style.set_caption("Impagos DataFrame")

display(styled_emails)
display(styled_clients)
display(styled_non_pay)

,id,fecha_envio,client_id,email
0,24,2022-06-12 06:23:15,1,"Hola Soy cliente pero no puedo acceder a mi factura on line y tampoco me puedo registrar como cliente porque siempre me sale una ventana que me dice ""error del servidor"" ¿Me pueden decir si hay alguna forma de consultar mis facturas? Podrían ponerlo un poco más fácil... Atentamente"
1,25,2022-08-12 19:15:39,2,"Hola, buenos días. Necesito que alguien me ayude para poder ver mi factura. No me deja crear correo. Gracias. Cuando puedan, ayúdenme."
2,26,2022-05-11 06:26:11,3,Cada cuanto facturas el gas. Llevo con vosotros desde el 10 de febrero y no me a llegado ninguna factura. Enviado desde Outlook para Android
3,27,2022-04-15 19:13:26,4,"Buenos días, No consigo ver la siguiente factura. Por favor enviarlo adjunto, gracias Factura Fecha Cups Importe Ver F-23-XXXXXXX 21/03/2023 ES00000054053403 490,90 €"
4,28,2022-04-07 19:25:24,5,"Hola buenas tardes, seguimos desde que realizaron cambios en su web, sin poder conectarnos y por tanto sin obtener nuestras facturas ni tampoco adjuntar las lecturas mensuales. Hemos solicitado en repetidas ocasiones y por diferentes medios que solventen el problema y todavía no tenemos acceso y no hemos recibido ninguna contestación al respecto. De nuevo nos vemos obligados a enviarles nuestras lecturas de GAS y LUZ por correo electrónico y solicitamos de nuevo las dos facturas (en pdf) que ya han sido pagadas por nuestra entidad bancaria y que no tenemos. FACTURAS : GAS por importe de 11,82 € y LUZ por importe de 153,49 €. Así mismo adjuntamos nuestras lecturas tanto de gas como de luz, para la confección de nuestra factura de marzo. Saludos Admin"
5,29,2022-02-18 19:29:31,6,"Porque motivo no puedo visualizar mi contrato después de firmar.Soy el firmante.Antonio Mel Lema,con DNI 12345678Z .Melo pueden enviar por correo electrónico.Gracias"
6,30,2022-05-12 20:01:26,7,"Me ha entrado la factura, ou sea, me ha entrado la factura, no, me ha llegado el cargo al banco de la luz y sin embargo no tengo las facturas. A ver si me las pueden mandar por correo, por favor. Vale, gracias a dios."
7,31,2022-05-08 08:54:34,8,"Hola, buenos días. Mira que me habéis cobrado las facturas de la luz y el gas, pero no me habéis mandado la factura. ¿Vale? Gracias."
8,32,2022-05-23 19:11:41,9,"Buenos dias , nos ponemos en contacto con ustedes para solicitar documentación necesaria para la central : TITULAR : XXXX PRUEBA NIF B123456789 REF CONTRATO :XXXXXXXX Contrato de electricidad vigente, Facturas de energía de los últimos 12 meses, Curvas de consumo del último año Quedamos a la espera , saludos!"
9,33,2022-08-08 20:42:46,2,"Buenos días, quería saber si tengo con vosotros contratado el gas. Gracias"


,id,nombre,email
0,1,Juan,juan@test.com
1,2,Pedro,pedro@test.com
2,3,Maria,maria@test.com
3,4,Jose,jose@test.com


,id,client_id,fecha_impago
0,1,1,2021-01-01
1,2,1,2021-02-01
2,3,8,2021-01-01
3,4,8,2021-02-01


### Create the model

**Categories analized:**

0. Solicitudes de ayuda / contacto urgente
1. Reenvío de facturas
2. Consulta contratos y documentación
3. Problemas con el área de clientes y problemas técnicos
4. Cambios de documentación / titularidad / bajas (cambio papeleos)
5. Reclamaciones y quejas (no hay, pero se puede añadir)
6. Consultas generales

In [6]:
label_mapping = {
    0: "Solicitudes de ayuda / contacto",
    1: "Reenvío de facturas",
    2: "Consulta contratos y documentación",
    3: "Problemas con el área de clientes y problemas técnicos",
    4: "Cambios de documentación / titularidad / bajas (cambio papeleos)",
    5: "Reclamaciones y quejas",
    6: "Consultas generales"
}

In [7]:
y_labels = [
    3, 3, 6, 1, 3, 2, 1, 1, 1, 2, 2, 2, 2, 4, 4, 1, 1, 1, 1, 1, 1, 1, 0
]

In [8]:
#Create the dataframe
emails = emails_df['email'].tolist()
data = {'Category': y_labels, 'Email': emails_df['email'].tolist()}
emails_df = pd.DataFrame(data)

In [9]:
#Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
tokenized_texts = [tokenizer.tokenize(email) for email in emails]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

In [10]:
len(tokenized_texts)

23

In [11]:
X_train, X_test, y_train, y_test = train_test_split(emails, y_labels, test_size=0.2, random_state=42)

In [12]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

#Tokenize and encode emails
train_encodings = tokenizer(X_train, truncation = True, padding = True  )
test_encodings = tokenizer(X_test, truncation = True, padding = True )

In [13]:
#Convert into tensors and obtain input_ids and attention_mask
input_ids_train = torch.tensor(train_encodings['input_ids'])
labels_train = torch.tensor(y_train)
attention_mask_train = torch.tensor(train_encodings['attention_mask'])

input_ids_test = torch.tensor(test_encodings['input_ids'])
labels_test = torch.tensor(y_test)
attention_mask_test = torch.tensor(test_encodings['attention_mask'])

In [14]:
# Create DataLoaders
dataset_train = TensorDataset(input_ids_train, attention_mask_train, labels_train)
dataloader_train = DataLoader(dataset_train, batch_size=4, shuffle=True)

dataset_test = TensorDataset(input_ids_test, attention_mask_test, labels_test)
dataloader_test = DataLoader(dataset_test, batch_size=4, shuffle=True)

In [15]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=len(label_mapping))

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
BERT_MODEL_NAME = 'distilbert-base-multilingual-cased'
LEARNING_RATE = 2e-5
EPOCHS = 3

In [17]:
# Fine-Tuning
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.CrossEntropyLoss()

#Train the model
for epoch in range(EPOCHS):
    for batch_input_ids, batch_attention_mask, batch_labels in dataloader_train:
        optimizer.zero_grad()
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, batch_labels)
        loss.backward()
        optimizer.step()

In [18]:
# Evaluation (on test set)
model.eval()
correct = 0
total = 0
predictions = []
true_labels = []

for batch_input_ids, batch_attention_mask, batch_labels in dataloader_test:
    with torch.no_grad():
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
        predictions.extend(predicted.tolist())
        true_labels.extend(batch_labels.tolist())

#Compute total accuracy of the model
accuracy = correct / total
print('Test Accuracy: {:.2f}%'.format(100 * accuracy))

# Generate classification report
report = classification_report(true_labels, predictions)
print(report)

Test Accuracy: 60.00%
              precision    recall  f1-score   support

           1       0.60      1.00      0.75         3
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1

    accuracy                           0.60         5
   macro avg       0.20      0.33      0.25         5
weighted avg       0.36      0.60      0.45         5



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
email_text = "Necesito la factura del dia 06/10/2022."

# Tokenize and encode the new email text
tokenized_text = tokenizer.tokenize(email_text)
input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
input_ids = torch.tensor(input_ids).unsqueeze(0)
attention_mask = torch.ones_like(input_ids)

# Make prediction
model.eval() 
with torch.no_grad(): 
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

predicted_label = label_mapping[predicted_class]

print("Predicted label:", predicted_label)

Predicted label: Reenvío de facturas


In [20]:
model_folder = '/content/drive/MyDrive/prueba/model/'
model.save_pretrained(model_folder)
tokenizer.save_pretrained(model_folder)

('/content/drive/MyDrive/prueba/model/tokenizer_config.json',
 '/content/drive/MyDrive/prueba/model/special_tokens_map.json',
 '/content/drive/MyDrive/prueba/model/vocab.txt',
 '/content/drive/MyDrive/prueba/model/added_tokens.json')